In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
pistons = MoneyLine(event="pistons", bet_amount=100, odds=235)
phi76 = MoneyLine(event="phi76", bet_amount=100, odds=-286)

raptors = MoneyLine(event="raptors", bet_amount=100, odds=180)
pacers = MoneyLine(event="pacers", bet_amount=100, odds=-220)

wizards = MoneyLine(event="wizards", bet_amount=100, odds=132)
knicks = MoneyLine(event="knicks", bet_amount=100, odds=-157)

bulls = MoneyLine(event="bulls", bet_amount=100, odds=163)
magic = MoneyLine(event="magic", bet_amount=100, odds=-195)

hawks = MoneyLine(event="hawks", bet_amount=100, odds=130)
cavs = MoneyLine(event="cavs", bet_amount=100, odds=-159)


binaries = [
#     [pistons, phi76],
#     [raptors, pacers],
    [wizards, knicks],
    [bulls, magic],
    [hawks, cavs]
]

results = [
#     [0, 1], #[pistons, phi76],
#     [0, 1], #[raptors, pacers],
    [0, 1], #[wizards, knicks],
    [0, 1], #[bulls, magic],
    [0, 1], #[hawks, cavs]
]

override_arr = [0, 2, 99]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)

In [3]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [4]:
len(all_subsets)

1

In [5]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.15,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup,
                    override_arr=override_arr
                    )
    
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

yoyoyoyoyo [{'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x11f0a10e0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<listcomp>.<lambda> at 0x11f0a1710>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<listcomp>.<lambda> at 0x11f0a17a0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<listcomp>.<lambda> at 0x11f0a1830>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<listcomp>.<lambda> at 0x11f0a18c0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<listcomp>.<lambda> at 0x11f0a1950>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<listcomp>.<lambda> at 0x11f0a19e0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<listcomp>.<lambda> at 0x11f0a1a70>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<listcomp>.<lambda> at 0x11f0a1b00>}]
     fun: -0.23927261540423506
     

In [6]:
df.describe()

,odds,bet,mult,payout,profit
count,8.000000,8.000000,8.000000,8.000000,8.000000
mean,705.082286,0.624875,8.050823,4.614288,-0.385625
std,318.462456,0.151181,3.184625,0.656182,0.655938
min,303.388502,0.367000,4.033885,3.423600,-1.576000
25%,471.177738,0.519500,5.711777,4.127350,-0.872250
50%,654.260841,0.650000,7.542608,4.895250,-0.105000
75%,891.117075,0.722250,9.911171,5.150000,0.150000
max,1303.368000,0.849000,14.033680,5.150000,0.150000


In [7]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,odds,bet,mult,payout,profit
count,4.000000,4.000000,4.000000,4.000000,4.00000
mean,461.488553,0.740000,5.614886,4.086700,-0.91300
std,122.075578,0.077395,1.220756,0.512002,0.51185
min,303.388502,0.666000,4.033885,3.423600,-1.57600
25%,428.026204,0.708000,5.280262,3.946350,-1.05325
50%,470.642527,0.722500,5.706425,4.125100,-0.87450
75%,504.104876,0.754500,6.041049,4.265450,-0.73425
max,601.280655,0.849000,7.012807,4.673000,-0.32700


In [8]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,odds,bet,mult,payout,profit
count,4.000000,4.000000,4.000000,4.000000,4.00000
mean,948.676019,0.509750,10.486760,5.141875,0.14175
std,252.003807,0.109527,2.520038,0.016250,0.01650
min,707.241026,0.367000,8.072410,5.117500,0.11700
25%,844.450225,0.480250,9.444502,5.141875,0.14175
50%,892.047526,0.519000,9.920475,5.150000,0.15000
75%,996.273321,0.548500,10.962733,5.150000,0.15000
max,1303.368000,0.634000,14.033680,5.150000,0.15000


In [9]:
negative_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
7,knicks_magic_cavs,"[1, 3, 5]","[1, 1, 1]",True,303.388502,0.849,4.033885,3.4236,-1.576
6,knicks_magic_hawks,"[1, 3, 4]","[1, 1, 0]",False,469.572105,0.723,5.695721,4.1206,-0.879
3,wizards_magic_cavs,"[0, 3, 5]","[0, 1, 1]",False,471.712950,0.722,5.717129,4.1296,-0.870
5,knicks_bulls_cavs,"[1, 2, 5]","[1, 0, 1]",False,601.280655,0.666,7.012807,4.6730,-0.327


In [10]:
positive_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,wizards_bulls_hawks,"[0, 2, 4]","[0, 0, 0]",False,1303.368000,0.367,14.033680,5.1500,0.150
1,wizards_bulls_cavs,"[0, 2, 5]","[0, 0, 1]",False,893.908428,0.518,9.939084,5.1500,0.150
4,knicks_bulls_hawks,"[1, 2, 4]","[1, 0, 0]",False,890.186624,0.520,9.901866,5.1500,0.150
2,wizards_magic_hawks,"[0, 3, 4]","[0, 1, 0]",False,707.241026,0.634,8.072410,5.1175,0.117


In [11]:
positive_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
False,4,4,4,4,4,4,4,4


In [12]:
negative_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
False,3,3,3,3,3,3,3,3
True,1,1,1,1,1,1,1,1


In [13]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,knicks_magic_cavs,"['1', '3', '5']","[1, 1, 1]",True,303.389,0.849,4.03389,3.4236,-1.576
1,knicks_magic_hawks,"['1', '3', '4']","[1, 1, 0]",False,469.572,0.723,5.69572,4.1206,-0.879
2,wizards_magic_cavs,"['0', '3', '5']","[0, 1, 1]",False,471.713,0.722,5.71713,4.1296,-0.87
3,knicks_bulls_cavs,"['1', '2', '5']","[1, 0, 1]",False,601.281,0.666,7.01281,4.673,-0.327


In [14]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,wizards_bulls_hawks,"['0', '2', '4']","[0, 0, 0]",False,1303.37,0.367,14.0337,5.15,0.15
1,wizards_bulls_cavs,"['0', '2', '5']","[0, 0, 1]",False,893.908,0.518,9.93908,5.15,0.15
2,knicks_bulls_hawks,"['1', '2', '4']","[1, 0, 0]",False,890.187,0.52,9.90187,5.15,0.15
3,wizards_magic_hawks,"['0', '3', '4']","[0, 1, 0]",False,707.241,0.634,8.07241,5.1175,0.117


In [15]:
sf49ers = MoneyLine(event="parlay", bet_amount=100, odds=1002)
sf49ers.print_stats()

    event  bet_amount  odds  multiplier  payout
0  parlay         100  1002       11.02  1102.0


In [16]:
pos = MoneyLine(event="pos", bet_amount=100, odds=-1020)
pos.print_stats()

  event  bet_amount  odds  multiplier      payout
0   pos         100 -1020    1.098039  109.803922


In [17]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-305)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -305    1.327869  1.327869


In [18]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-285)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -285    1.350877  1.350877
